<a href="https://colab.research.google.com/github/deepika1715/Artificial-Intelligence/blob/main/Detecting_deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install keras
!pip install efficientnet
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


In [12]:
from google.colab import files

# Upload kaggle.json
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"deepika1715","key":"9e13b3074137a4ad61588a8112ef7c5d"}'}

In [9]:
#making .kaggle directory under root
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [13]:
#moving kaggle.json to .kaggle directory
!mv kaggle.json ~/.kaggle/

In [14]:
#changing access permision for the json file
!chmod 600 ~/.kaggle/kaggle.json

In [15]:
import kaggle

In [16]:
#importing required libraries
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import scikitplot as skplt

In [30]:
#downloading the dataset
#!kaggle datasets download -d fakecatcherai/dfdc-dataset
!kaggle competitions download -c deepfake-detection-challenge




100% 4.12G/4.13G [00:43<00:00, 253MB/s]
100% 4.13G/4.13G [00:43<00:00, 102MB/s]


In [32]:
!unzip /root/kaggle/deepfake-detection-challenge.zip -d /root/kaggle/DFDC/


Archive:  /root/kaggle/deepfake-detection-challenge.zip
  inflating: /root/kaggle/DFDC/sample_submission.csv  
  inflating: /root/kaggle/DFDC/test_videos/aassnaulhq.mp4  
  inflating: /root/kaggle/DFDC/test_videos/aayfryxljh.mp4  
  inflating: /root/kaggle/DFDC/test_videos/acazlolrpz.mp4  
  inflating: /root/kaggle/DFDC/test_videos/adohdulfwb.mp4  
  inflating: /root/kaggle/DFDC/test_videos/ahjnxtiamx.mp4  
  inflating: /root/kaggle/DFDC/test_videos/ajiyrjfyzp.mp4  
  inflating: /root/kaggle/DFDC/test_videos/aktnlyqpah.mp4  
  inflating: /root/kaggle/DFDC/test_videos/alrtntfxtd.mp4  
  inflating: /root/kaggle/DFDC/test_videos/aomqqjipcp.mp4  
  inflating: /root/kaggle/DFDC/test_videos/apedduehoy.mp4  
  inflating: /root/kaggle/DFDC/test_videos/apvzjkvnwn.mp4  
  inflating: /root/kaggle/DFDC/test_videos/aqrsylrzgi.mp4  
  inflating: /root/kaggle/DFDC/test_videos/axfhbpkdlc.mp4  
  inflating: /root/kaggle/DFDC/test_videos/ayipraspbn.mp4  
  inflating: /root/kaggle/DFDC/test_videos/bcbqxh

In [33]:
#Load the DFDC Dataset:Assume the dataset contains video files. You need to extract frames from videos to train the model.


import os
import cv2
import glob
import numpy as np
from tqdm import tqdm

def extract_frames(video_path, output_dir, frames_per_video=5):
    os.makedirs(output_dir, exist_ok=True)
    video_files = glob.glob(video_path + "/*.mp4")

    for video_file in tqdm(video_files):
        cap = cv2.VideoCapture(video_file)
        video_name = os.path.splitext(os.path.basename(video_file))[0]

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        selected_frames = np.linspace(0, frame_count-1, frames_per_video, dtype=int)

        for i, frame_num in enumerate(selected_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            success, frame = cap.read()
            if success:
                frame_filename = f"{video_name}_frame_{i}.jpg"
                cv2.imwrite(os.path.join(output_dir, frame_filename), frame)

        cap.release()

# Example usage
extract_frames('/root/kaggle/DFDC/train', '/root/kaggle/DFDC/train_frames')


0it [00:00, ?it/s]


In [22]:

!unzip /root/kaggle/dfdc-dataset.zip -d /root/kaggle/DFDC/

Archive:  /root/kaggle/dfdc-dataset.zip
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aassnaulhq_frame0.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aassnaulhq_frame174.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aassnaulhq_frame261.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aassnaulhq_frame87.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aayfryxljh_frame0.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aayfryxljh_frame174.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aayfryxljh_frame261.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/aayfryxljh_frame87.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/acazlolrpz_frame0.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/acazlolrpz_frame174.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/acazlolrpz_frame261.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/acazlolrpz_frame87.jpg  
  inflating: /root/kaggle/DFDC/DFDC/test/fake/adohdulfwb_frame0.jpg  
  inflating: /root/kaggle/DFDC/DFDC

In [70]:

import cv2
import os
import json

def extract_frames(video_path, output_folder, frame_rate=1):
    video_capture = cv2.VideoCapture(video_path)
    count = 0
    success, image = video_capture.read()
    while success:
        if count % frame_rate == 0:
            cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), image)
        success, image = video_capture.read()
        count += 1
    video_capture.release()

# Example usage:
extract_frames('/root/kaggle/DFDC/test_videos', '/root/kaggle/DFDC/test_frames')




In [71]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory structure
train_dir = '/root/kaggle/DFDC/train_frames'
val_dir = '/root/kaggle/DFDC/train_sample_videos'

# Example data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [72]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load EfficientNetB0 with pre-trained weights on ImageNet and exclude the top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # Adding Dropout for regularization
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# Combine base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

In [73]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [90]:
#extracting train frames

def extract_frames_from_videos(video_dir, output_dir, frame_rate=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all video files in the video_dir
    video_files = [f for f in os.listdir(video_dir) if os.path.isfile(os.path.join(video_dir, f))]

    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        output_folder = os.path.join(output_dir, os.path.splitext(video_file)[0])  # Create a folder for each video
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        video_capture = cv2.VideoCapture(video_path)
        count = 0
        success, image = video_capture.read()
        while success:
            if count % frame_rate == 0:
                cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), image)
            success, image = video_capture.read()
            count += 1
        video_capture.release()
        print(f'Frames extracted from {video_file} to {output_folder}')

# Example usage:
extract_frames_from_videos('/root/kaggle/DFDC/train_sample_videos', '/root/kaggle/DFDC/train_frames')


Frames extracted from dcuiiorugd.mp4 to /root/kaggle/DFDC/train_frames/dcuiiorugd
Frames extracted from atkdltyyen.mp4 to /root/kaggle/DFDC/train_frames/atkdltyyen
Frames extracted from cizlkenljw.mp4 to /root/kaggle/DFDC/train_frames/cizlkenljw
Frames extracted from bdnaqemxmr.mp4 to /root/kaggle/DFDC/train_frames/bdnaqemxmr
Frames extracted from alninxcyhg.mp4 to /root/kaggle/DFDC/train_frames/alninxcyhg
Frames extracted from dsndhujjjb.mp4 to /root/kaggle/DFDC/train_frames/dsndhujjjb
Frames extracted from adylbeequz.mp4 to /root/kaggle/DFDC/train_frames/adylbeequz
Frames extracted from bwhlgysghg.mp4 to /root/kaggle/DFDC/train_frames/bwhlgysghg
Frames extracted from cknyxaqouy.mp4 to /root/kaggle/DFDC/train_frames/cknyxaqouy
Frames extracted from aipfdnwpoo.mp4 to /root/kaggle/DFDC/train_frames/aipfdnwpoo
Frames extracted from bguwlyazau.mp4 to /root/kaggle/DFDC/train_frames/bguwlyazau
Frames extracted from dvumqqhoac.mp4 to /root/kaggle/DFDC/train_frames/dvumqqhoac
Frames extracted

KeyboardInterrupt: 

In [74]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory structure - Verify that these directories exist and contain images
train_dir = '/root/kaggle/DFDC/train_frames'
val_dir = '/root/kaggle/DFDC/train_sample_videos'

# Print the number of images found in each directory
print("Number of images in training directory:", len(os.listdir(train_dir)))
print("Number of images in validation directory:", len(os.listdir(val_dir)))

# Example data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Check if generators are yielding data
print("Training generator yields data:", bool(next(train_generator, None)))
print("Validation generator yields data:", bool(next(validation_generator, None)))

Number of images in training directory: 2
Number of images in validation directory: 401
Found 0 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Training generator yields data: True
Validation generator yields data: True


In [89]:
#extracting test frames
import cv2
import os

def extract_frames_from_videos(video_dir, output_dir, frame_rate=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all video files in the video_dir
    video_files = [f for f in os.listdir(video_dir) if os.path.isfile(os.path.join(video_dir, f))]

    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        output_folder = os.path.join(output_dir, os.path.splitext(video_file)[0])  # Create a folder for each video
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        video_capture = cv2.VideoCapture(video_path)
        count = 0
        success, image = video_capture.read()
        while success:
            if count % frame_rate == 0:
                cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), image)
            success, image = video_capture.read()
            count += 1
        video_capture.release()
        print(f'Frames extracted from {video_file} to {output_folder}')

# Example usage:
extract_frames_from_videos('/root/kaggle/DFDC/test_videos', '/root/kaggle/DFDC/test_frames')



Frames extracted from sylnrepacf.mp4 to /root/kaggle/DFDC/test_frames/sylnrepacf
Frames extracted from ybbrkacebd.mp4 to /root/kaggle/DFDC/test_frames/ybbrkacebd
Frames extracted from vizerpsvbz.mp4 to /root/kaggle/DFDC/test_frames/vizerpsvbz
Frames extracted from wixbuuzygv.mp4 to /root/kaggle/DFDC/test_frames/wixbuuzygv
Frames extracted from apedduehoy.mp4 to /root/kaggle/DFDC/test_frames/apedduehoy
Frames extracted from kvmpmhdxly.mp4 to /root/kaggle/DFDC/test_frames/kvmpmhdxly
Frames extracted from ncmpqwmnzb.mp4 to /root/kaggle/DFDC/test_frames/ncmpqwmnzb
Frames extracted from ryxaqpfubf.mp4 to /root/kaggle/DFDC/test_frames/ryxaqpfubf
Frames extracted from iorbtaarte.mp4 to /root/kaggle/DFDC/test_frames/iorbtaarte
Frames extracted from jyoxdvxpza.mp4 to /root/kaggle/DFDC/test_frames/jyoxdvxpza
Frames extracted from tejfudfgpq.mp4 to /root/kaggle/DFDC/test_frames/tejfudfgpq
Frames extracted from usqqvxcjmg.mp4 to /root/kaggle/DFDC/test_frames/usqqvxcjmg
Frames extracted from hxwtsa

KeyboardInterrupt: 

In [79]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory structure - Verify that these directories exist and contain images
train_dir = '/root/kaggle/DFDC/train_frames'
val_dir = '/root/kaggle/DFDC/train_sample_videos'
test_dir = '/root/kaggle/DFDC/test_frames' # Assuming you have a test directory

# ... (rest of the code remains the same)

# Create a test generator
test_datagen = ImageDataGenerator(rescale=1./255) # You might

In [80]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory containing test data
test_dir = '/root/kaggle/DFDC/test_frames'

# Create a data generator for test data
test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for testing

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Assuming binary classification
    shuffle=False  # Important for evaluation to ensure predictions correspond to the correct labels
)


Found 13809 images belonging to 47 classes.


In [81]:
# Verify test_generator
print(test_generator.class_indices)  # Check class indices
print(len(test_generator.filenames))  # Check number of test samples


{'aktnlyqpah': 0, 'apedduehoy': 1, 'bkuzquigyt': 2, 'blnmxntbey': 3, 'bofrwgeyjo': 4, 'cnxccbjlct': 5, 'etdliwticv': 6, 'frqfsucgao': 7, 'gfcycflhbo': 8, 'gfgcwxkbjd': 9, 'gpsxfxrjrr': 10, 'hxwtsaydal': 11, 'iorbtaarte': 12, 'jhczqfefgw': 13, 'jsysgmycsx': 14, 'jyoxdvxpza': 15, 'kcjvhgvhpt': 16, 'kmcdjxmnoa': 17, 'kvmpmhdxly': 18, 'lebzjtusnr': 19, 'ljauauuyka': 20, 'mdfndlljvt': 21, 'ncmpqwmnzb': 22, 'nxgzmgzkfv': 23, 'pxcfrszlgi': 24, 'qcbkztamqc': 25, 'qlvsqdroqo': 26, 'qsjiypnjwi': 27, 'ryxaqpfubf': 28, 'sylnrepacf': 29, 'tejfudfgpq': 30, 'tyjpjpglgx': 31, 'udxqbhgvvx': 32, 'usqqvxcjmg': 33, 'vizerpsvbz': 34, 'vokrpfjpeb': 35, 'vssmlqoiti': 36, 'wcqvzujamg': 37, 'wixbuuzygv': 38, 'wvgviwnwob': 39, 'xdezcezszc': 40, 'xphdfgmfmz': 41, 'ybbrkacebd': 42, 'yllztsrwjw': 43, 'ystdtnetgj': 44, 'zfobicuigx': 45, 'zgbhzkditd': 46}
13809


In [82]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [83]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 13809 images belonging to 47 classes.


In [84]:
# Evaluate on a batch from the generator
x_batch, y_batch = next(test_generator)
test_loss, test_accuracy = model.evaluate(x_batch, y_batch)
print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.0000e+00 - loss: 0.6933
Test loss: 0.69, Test accuracy: 0.00


In [85]:
# Verify if test_generator is empty
if len(test_generator.filenames) == 0:
    print("Error: No test samples found in the directory.")
else:
    # Evaluate on a batch from the generator
    x_batch, y_batch = next(test_generator)
    test_loss, test_accuracy = model.evaluate(x_batch, y_batch)
    print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.6933
Test loss: 0.69, Test accuracy: 0.00


In [86]:
folder_path = '/root/kaggle/DFDC/test_frames'
shutil.rmtree(folder_path)

In [95]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}')

# Predict on test data
test_generator.reset()
predictions = model.predict(test_generator)
y_pred = (predictions > 0.5).astype(int)

# Get true labels
y_true = test_generator.classes

# Compute confusion matrix and classification report
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(cr)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Plot accuracy and loss curves
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss')

plt.show()


NameError: name 'y_pred' is not defined